In [2]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!unzip "/content/training.zip"

In [5]:
!unzip "/content/test.zip"

Archive:  /content/test.zip
   creating: test/
  inflating: test/1.jpg              
  inflating: test/10.jpg             
  inflating: test/11.jpg             
  inflating: test/12.jpg             
  inflating: test/13.jpg             
  inflating: test/14.jpg             
  inflating: test/15.jpg             
  inflating: test/16.jpg             
  inflating: test/17.jpg             
  inflating: test/18.jpg             
  inflating: test/19.jpg             
  inflating: test/2.jpg              
  inflating: test/20.jpg             
  inflating: test/21.jpg             
  inflating: test/22.jpg             
  inflating: test/23.jpg             
  inflating: test/24.jpg             
  inflating: test/25.jpg             
  inflating: test/26.jpg             
  inflating: test/27.jpg             
  inflating: test/28.jpg             
  inflating: test/29.jpg             
  inflating: test/3.jpg              
  inflating: test/30.jpg             
  inflating: test/31.jpg             
  i

In [ ]:
shutil.rmtree('/content/full_test') #deletes a directory and all its contents.

In [6]:
path = "/content/training/background"

target_train = "/content/full_train"
target_test = "/content/full_test"

os.mkdir(target_train)
os.mkdir(target_test)

target_train_neg = "/content/full_train/neg"
target_test_neg = "/content/full_test/neg"

os.mkdir(target_train_neg)
os.mkdir(target_test_neg)


In [7]:
path2 = "/content/training/hi"

target_train_pos = "/content/full_train/pos"
target_test_pos = "/content/full_test/pos"

os.mkdir(target_train_pos)
os.mkdir(target_test_pos)

In [8]:
import random
image_names_1 = os.listdir(path)
random.shuffle(image_names_1)
image_names_2 = os.listdir(path2)
random.shuffle(image_names_2)

In [9]:
p, dirs, files = next(os.walk(path))
file_count_1 = len(files)
print(file_count_1)
p, dirs, files = next(os.walk(path2))
file_count_2 = len(files)
print(file_count_2)

4450
1425


In [10]:
import math
train = math.floor(file_count_1*0.8)
test = file_count_1 - train
print(train,test)

3560 890


In [11]:
import shutil
j=0
for i in range(file_count_1):
    
    image_name = image_names_1[i]
    image_path = os.path.join(path,image_name)
    
    if(i<train):
        target_path = os.path.join(target_train_neg,image_name)
    else:
        j=j+1
        target_path = os.path.join(target_test_neg,image_name)
    
    shutil.copy2(image_path,target_path)
print(j)

890


In [12]:
train = math.floor(file_count_2*0.8)
test = file_count_2 - train
print(train,test)

1140 285


In [13]:
j=0
for i in range(file_count_2):
    
    image_name = image_names_2[i]
    image_path = os.path.join(path2,image_name)
    
    if(i<train):
        target_path = os.path.join(target_train_pos,image_name)
    else:
        j=j+1
        target_path = os.path.join(target_test_pos,image_name)
    
    shutil.copy2(image_path,target_path)
print(j)

285


In [14]:
train_datagen = ImageDataGenerator(rescale = 1/255., width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.1, zoom_range = 0.1, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1/255.)

train_data = train_datagen.flow_from_directory(target_train, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

test_data = test_datagen.flow_from_directory(target_test, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 4700 images belonging to 2 classes.
Found 1175 images belonging to 2 classes.


In [15]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

base_model = ResNet50V2(include_top=False)
base_model.trainable = False

94683136/94668760 [==============================] - 1s 0us/step


In [16]:
input = tf.keras.Input(shape = (224,224,3))

x = base_model(input, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(1, activation= 'sigmoid')(x)

model_0 = tf.keras.Model(input, output)

In [17]:
model_0.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [18]:
model_0.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [19]:
history_model_0 = model_0.fit(train_data,
                              epochs = 10,
                              steps_per_epoch = len(train_data),
                              validation_data = test_data,
                              validation_steps = 0.25*len(test_data),
                              )

Epoch 1/10
235/235 [==============================] - 101s 284ms/step - loss: 0.3128 - accuracy: 0.8732 - val_loss: 0.2006 - val_accuracy: 0.9233
Epoch 2/10
235/235 [==============================] - 66s 279ms/step - loss: 0.2132 - accuracy: 0.9132 - val_loss: 0.1924 - val_accuracy: 0.9433
Epoch 3/10
235/235 [==============================] - 65s 277ms/step - loss: 0.1856 - accuracy: 0.9223 - val_loss: 0.2109 - val_accuracy: 0.9200
Epoch 4/10
235/235 [==============================] - 66s 282ms/step - loss: 0.1687 - accuracy: 0.9374 - val_loss: 0.1794 - val_accuracy: 0.9300
Epoch 5/10
235/235 [==============================] - 68s 287ms/step - loss: 0.1625 - accuracy: 0.9370 - val_loss: 0.1933 - val_accuracy: 0.9233
Epoch 6/10
235/235 [==============================] - 67s 285ms/step - loss: 0.1512 - accuracy: 0.9402 - val_loss: 0.1749 - val_accuracy: 0.9233
Epoch 7/10
235/235 [==============================] - 67s 285ms/step - loss: 0.1498 - accuracy: 0.9447 - val_loss: 0.1458 - val_a

In [20]:
model_0.evaluate(test_data)

59/59 [==============================] - 10s 161ms/step - loss: 0.1532 - accuracy: 0.9447


[0.15315529704093933, 0.9446808695793152]

In [21]:
img = tf.io.read_file("/content/1.jpg")
img = tf.image.decode_image(img)
img = tf.image.resize(img, size = [224,224])
img = img/255.

In [ ]:
img

In [62]:
list1=os.listdir(test_d)

k=[]    
for item in list1:    
    head, sep, tail = item.partition('.')
    k.append(int(head))

k.sort()
print(k)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]


In [ ]:
test_d = "/content/test"

In [67]:
predictions = []
for i in k:
  img = tf.io.read_file(test_d + '/' + str(i) + '.jpg')
  img = tf.image.decode_image(img)
  img = tf.image.resize(img, size = [224,224])
  img = img/255.
  prediction = model_0.predict(tf.expand_dims(img, axis = 0))
  predictions.append(prediction)

In [68]:
predictions, len(predictions)

([array([[0.9225697]], dtype=float32),
  array([[0.01588476]], dtype=float32),
  array([[0.9782574]], dtype=float32),
  array([[0.02264301]], dtype=float32),
  array([[0.9765667]], dtype=float32),
  array([[0.00034441]], dtype=float32),
  array([[0.909128]], dtype=float32),
  array([[0.0150571]], dtype=float32),
  array([[0.55588573]], dtype=float32),
  array([[0.37531483]], dtype=float32),
  array([[0.93622416]], dtype=float32),
  array([[0.16365027]], dtype=float32),
  array([[0.99999213]], dtype=float32),
  array([[0.01656307]], dtype=float32),
  array([[0.98950154]], dtype=float32),
  array([[0.023779]], dtype=float32),
  array([[0.11105779]], dtype=float32),
  array([[0.03544933]], dtype=float32),
  array([[0.71080947]], dtype=float32),
  array([[0.76991427]], dtype=float32),
  array([[0.9955663]], dtype=float32),
  array([[0.99890757]], dtype=float32),
  array([[0.9999949]], dtype=float32),
  array([[0.9966883]], dtype=float32),
  array([[0.92060184]], dtype=float32),
  array([[0

In [74]:
import random
import json
# from utils.io import write_json

def write_json(filename, result):
    with open(filename, 'w') as outfile:
        json.dump(result, outfile)

def read_json(filename):
    with open(filename, 'r') as outfile:
        data =  json.load(outfile)
    return data

def generate_sample_file(filename):
    res = {}
    for i in range(1,98):
        test_set = str(i) + '.png'
        if(predictions[i]>0.5):
          res[test_set] = 1
        else:
          res[test_set] = 0

    write_json(filename, res)

if __name__ == '__main__':
    generate_sample_file('./sample_result.json')

In [75]:
model_0.save("hindi_text_identification.h5")

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
